##  감정분석
- 나이브 베이즈 분류를 이용한 감정분석

**나이브 베이즈 분류**
> 기계 학습 분야에서, '나이브 베이즈 분류(Naïve Bayes Classification)는 특성들 사이의 독립을 가정하는 베이즈 정리를 적용한 확률 분류기의 일종으로 1950년대 이후 광범위하게 연구되고 있다.
통계 및 컴퓨터 과학 문헌에서, 나이브 베이즈는 단순 베이즈, 독립 베이즈를 포함한 다양한 이름으로 알려져 있으며, 1960년대 초에 텍스트 검색 커뮤니티에 다른 이름으로 소개되기도 하였다.
나이브 베이즈 분류는 텍스트 분류에 사용됨으로써 문서를 여러 범주 (예: 스팸, 스포츠, 정치) 중 하나로 판단하는 문제에 대한 대중적인 방법으로 남아있다. 또한, 자동 의료 진단 분야에서의 응용사례를 보면, 적절한 전처리를 하면 더 진보된 방법들 (예: 서포트 벡터 머신 (Support Vector Machine))과도 충분한 경쟁력을 보임을 알 수 있다.



### 영어

In [7]:
from nltk.tokenize import word_tokenize
import nltk 

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bigdata\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

- 나이브 베이즈 분류기는 지도학습이라서 정답을 알려주어야 한다.

In [3]:
train = [('i like you', 'pos'),
        ('i hate you', 'neg'),
        ('you like me', 'neg'),
        ('i like her', 'pos')]

- 전체 말뭉치 만들기

In [4]:
sentence = train[0]
word_tokenize(sentence[0])


['i', 'like', 'you']

In [5]:
# set 명령을 안 할 경우
[word.lower() for sentence in train for word in word_tokenize(sentence[0])]

['i',
 'like',
 'you',
 'i',
 'hate',
 'you',
 'you',
 'like',
 'me',
 'i',
 'like',
 'her']

In [6]:
# train 에서 하나씩 sentence를 받아오고 senetence의 [0] 첫번째를 토큰화해서 워드로 받아오고 이걸 소문자로 변형
# 집합으로 받아와서 중복되지 않게 가져옴.

all_words  = set(
    word.lower() for sentence in train for word in word_tokenize(sentence[0])
)
all_words

{'hate', 'her', 'i', 'like', 'me', 'you'}

In [7]:
for x in train:
    print(x)

('i like you', 'pos')
('i hate you', 'neg')
('you like me', 'neg')
('i like her', 'pos')


In [8]:
t =  [({word: (word in word_tokenize(x[0])) for word in all_words}, x[1]) for x in train ]
t

# 이때 t에서 위의 두개를 살펴보면 you 가 둘다 들어가있는 문장이 각각 pos, neg로 50%확률이다.
# 이렇게 독립적으로 사건이 일어나는 것을 나이브 베이즈 특성이라 할 수 있다.

[({'me': False,
   'you': True,
   'her': False,
   'i': True,
   'like': True,
   'hate': False},
  'pos'),
 ({'me': False,
   'you': True,
   'her': False,
   'i': True,
   'like': False,
   'hate': True},
  'neg'),
 ({'me': True,
   'you': True,
   'her': False,
   'i': False,
   'like': True,
   'hate': False},
  'neg'),
 ({'me': False,
   'you': False,
   'her': True,
   'i': True,
   'like': True,
   'hate': False},
  'pos')]

In [10]:
# 나이브 베이즈에서는 sklearn이 아니라 nltk 를 사용
#이렇게 각 단어별로 독립적으로 확률을 계산하기 때문에 naive하다고 한다.

clf = nltk.NaiveBayesClassifier.train(t)
clf.show_most_informative_features()

Most Informative Features
                    hate = False             pos : neg    =      1.7 : 1.0
                     her = False             neg : pos    =      1.7 : 1.0
                       i = True              pos : neg    =      1.7 : 1.0
                    like = True              pos : neg    =      1.7 : 1.0
                      me = False             pos : neg    =      1.7 : 1.0
                     you = True              neg : pos    =      1.7 : 1.0


In [12]:
test_sentence =  'i like MeRui'
test_sent_featues = {
    word.lower() : (word in word_tokenize(test_sentence.lower())) for word in all_words
}
test_sent_featues


{'me': False,
 'you': False,
 'her': False,
 'i': True,
 'like': True,
 'hate': False}

In [15]:
clf.classify(test_sent_featues)

'pos'

### 한글
- 한글은 형태소 분석을 해야함
- Okt 는 konlpy 라는 파이썬 라이브러리에서 제공하는 형태소 분석기 중 하나
- Open Korean Text 의 약자로, 원래는 트위터에서 개발한 한국어 형태소 분석기
- 형태소 분석, 품사태깅, 명사추출, 어근 추출과 같은 기능을 한다.

**❌형태소 분석 없이 진행한 경우**
- 제대로 감성분석이 되지 않음

In [1]:
from konlpy.tag import Okt
pos_tagger = Okt()

In [2]:
train = [
    ('메리가 좋아','pos'),
    ('고양이도 좋아','pos'),
    ('난 수업이 지루해','neg'),
    ('메리는 이쁜 고양이야','pos'),
    ('난 마치고 메리랑 놀거야','pos'),
]

In [8]:
all_words = set(
    word for sentence in train for word in word_tokenize(sentence[0])
)
all_words

{'고양이도',
 '고양이야',
 '난',
 '놀거야',
 '마치고',
 '메리가',
 '메리는',
 '메리랑',
 '수업이',
 '이쁜',
 '좋아',
 '지루해'}

In [10]:
t = [({word : (word in word_tokenize(x[0])) for word in all_words},x[1]) for x in train ]
t

[({'고양이도': False,
   '메리는': False,
   '좋아': True,
   '고양이야': False,
   '이쁜': False,
   '마치고': False,
   '메리가': True,
   '놀거야': False,
   '메리랑': False,
   '난': False,
   '지루해': False,
   '수업이': False},
  'pos'),
 ({'고양이도': True,
   '메리는': False,
   '좋아': True,
   '고양이야': False,
   '이쁜': False,
   '마치고': False,
   '메리가': False,
   '놀거야': False,
   '메리랑': False,
   '난': False,
   '지루해': False,
   '수업이': False},
  'pos'),
 ({'고양이도': False,
   '메리는': False,
   '좋아': False,
   '고양이야': False,
   '이쁜': False,
   '마치고': False,
   '메리가': False,
   '놀거야': False,
   '메리랑': False,
   '난': True,
   '지루해': True,
   '수업이': True},
  'neg'),
 ({'고양이도': False,
   '메리는': True,
   '좋아': False,
   '고양이야': True,
   '이쁜': True,
   '마치고': False,
   '메리가': False,
   '놀거야': False,
   '메리랑': False,
   '난': False,
   '지루해': False,
   '수업이': False},
  'pos'),
 ({'고양이도': False,
   '메리는': False,
   '좋아': False,
   '고양이야': False,
   '이쁜': False,
   '마치고': True,
   '메리가': False,
   '놀거야': True,
   '메리랑': True,
   '난': 

In [11]:
clf = nltk.NaiveBayesClassifier.train(t)
clf.show_most_informative_features()

Most Informative Features
                       난 = True              neg : pos    =      2.5 : 1.0
                      좋아 = False             neg : pos    =      1.5 : 1.0
                    고양이도 = False             neg : pos    =      1.1 : 1.0
                    고양이야 = False             neg : pos    =      1.1 : 1.0
                     놀거야 = False             neg : pos    =      1.1 : 1.0
                     마치고 = False             neg : pos    =      1.1 : 1.0
                     메리가 = False             neg : pos    =      1.1 : 1.0
                     메리는 = False             neg : pos    =      1.1 : 1.0
                     메리랑 = False             neg : pos    =      1.1 : 1.0
                      이쁜 = False             neg : pos    =      1.1 : 1.0


In [14]:
test_sentence = '난 수업이 마치면 메리랑 놀거야'

test_sent_featues = {word.lower(): word in word_tokenize(test_sentence.lower()) for word in all_words}
test_sent_featues

{'고양이도': False,
 '메리는': False,
 '좋아': False,
 '고양이야': False,
 '이쁜': False,
 '마치고': False,
 '메리가': False,
 '놀거야': True,
 '메리랑': True,
 '난': True,
 '지루해': False,
 '수업이': True}

In [15]:
clf.classify(test_sent_featues)

'neg'

**⭕형태소 분석을 한 경우**
- 형태소 분석을 한 후 품사를 단어 뒤에 붙여넣도록 함

In [17]:
def tokenize(doc):
    return ['/'.join(t) for t in pos_tagger.pos(doc, norm = True, stem = True)]

In [20]:
train_docs = [(tokenize(row[0]), row[1]) for row in train]
train_docs

[(['메리/Noun', '가/Josa', '좋다/Adjective'], 'pos'),
 (['고양이/Noun', '도/Josa', '좋다/Adjective'], 'pos'),
 (['난/Noun', '수업/Noun', '이/Josa', '지루하다/Adjective'], 'neg'),
 (['메리/Noun', '는/Josa', '이쁘다/Adjective', '고양이/Noun', '야/Josa'], 'pos'),
 (['난/Noun', '마치/Noun', '고/Josa', '메리/Noun', '랑/Josa', '놀다/Verb'], 'pos')]

In [22]:
tokens = [t for d in train_docs for t in d[0]]
tokens

['메리/Noun',
 '가/Josa',
 '좋다/Adjective',
 '고양이/Noun',
 '도/Josa',
 '좋다/Adjective',
 '난/Noun',
 '수업/Noun',
 '이/Josa',
 '지루하다/Adjective',
 '메리/Noun',
 '는/Josa',
 '이쁘다/Adjective',
 '고양이/Noun',
 '야/Josa',
 '난/Noun',
 '마치/Noun',
 '고/Josa',
 '메리/Noun',
 '랑/Josa',
 '놀다/Verb']

In [26]:
def term_exist(docs):
    return {word : (word in set(docs)) for word in tokens}

In [28]:
train_xy = [(term_exist(d),c) for d,c in train_docs]
train_xy

[({'메리/Noun': True,
   '가/Josa': True,
   '좋다/Adjective': True,
   '고양이/Noun': False,
   '도/Josa': False,
   '난/Noun': False,
   '수업/Noun': False,
   '이/Josa': False,
   '지루하다/Adjective': False,
   '는/Josa': False,
   '이쁘다/Adjective': False,
   '야/Josa': False,
   '마치/Noun': False,
   '고/Josa': False,
   '랑/Josa': False,
   '놀다/Verb': False},
  'pos'),
 ({'메리/Noun': False,
   '가/Josa': False,
   '좋다/Adjective': True,
   '고양이/Noun': True,
   '도/Josa': True,
   '난/Noun': False,
   '수업/Noun': False,
   '이/Josa': False,
   '지루하다/Adjective': False,
   '는/Josa': False,
   '이쁘다/Adjective': False,
   '야/Josa': False,
   '마치/Noun': False,
   '고/Josa': False,
   '랑/Josa': False,
   '놀다/Verb': False},
  'pos'),
 ({'메리/Noun': False,
   '가/Josa': False,
   '좋다/Adjective': False,
   '고양이/Noun': False,
   '도/Josa': False,
   '난/Noun': True,
   '수업/Noun': True,
   '이/Josa': True,
   '지루하다/Adjective': True,
   '는/Josa': False,
   '이쁘다/Adjective': False,
   '야/Josa': False,
   '마치/Noun': False,
   '고/Jo

In [32]:
clf = nltk.NaiveBayesClassifier.train(train_xy)
clf.show_most_informative_features()

Most Informative Features
                  난/Noun = True              neg : pos    =      2.5 : 1.0
                 메리/Noun = False             neg : pos    =      2.5 : 1.0
                고양이/Noun = False             neg : pos    =      1.5 : 1.0
            좋다/Adjective = False             neg : pos    =      1.5 : 1.0
                  가/Josa = False             neg : pos    =      1.1 : 1.0
                  고/Josa = False             neg : pos    =      1.1 : 1.0
                 놀다/Verb = False             neg : pos    =      1.1 : 1.0
                  는/Josa = False             neg : pos    =      1.1 : 1.0
                  도/Josa = False             neg : pos    =      1.1 : 1.0
                  랑/Josa = False             neg : pos    =      1.1 : 1.0


In [34]:
test_sentence = '난 수업이 마치면 메리랑 놀거야'

test_docs =pos_tagger.pos(test_sentence)
test_docs

[('난', 'Noun'),
 ('수업', 'Noun'),
 ('이', 'Josa'),
 ('마치', 'Noun'),
 ('면', 'Josa'),
 ('메리', 'Noun'),
 ('랑', 'Josa'),
 ('놀거야', 'Verb')]

In [36]:
test_sent_featues = {word : (word in tokens) for word in test_docs}
test_sent_featues

{('난', 'Noun'): False,
 ('수업', 'Noun'): False,
 ('이', 'Josa'): False,
 ('마치', 'Noun'): False,
 ('면', 'Josa'): False,
 ('메리', 'Noun'): False,
 ('랑', 'Josa'): False,
 ('놀거야', 'Verb'): False}

In [37]:
clf.classify(test_sent_featues)

'pos'